In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import LogisticRegression


In [0]:
# DataFrame Schema, that should be a Table schema
coviddeath_schema = StructType([
  StructField('UID', IntegerType()),
  StructField('iso2', StringType()),
  StructField('iso3', StringType()),
  StructField('code3', IntegerType()),
  StructField('FIPS', IntegerType()),
  StructField('Admin2', StringType()),
  StructField('Lat', DoubleType()),
  StructField('CombinedKey', StringType()),
  StructField('Date1', StringType()),
  StructField('Case', IntegerType()),
  StructField('Long', DoubleType()),
  StructField('Country', StringType()),
  StructField('Province', StringType()),
  StructField('Temp', StringType()),
  StructField('Date', StringType()),
  StructField('Day', StringType()),
  StructField('Year', StringType())
  ])

In [0]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('UScasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [0]:
data = coviddeath.select("Year","Date","Day", "Temp", "Admin2","Lat","Long","Province",((col("Case") > 2).cast("Double").alias("label")))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)

data.show(5)


+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
2020| 22| 1| 61|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
2020| 23| 1| 59|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
2020| 24| 1| 63|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
2020| 25| 1| 64|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
2020| 26| 1| 58|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
only showing top 5 rows

In [0]:
splits = data.randomSplit([0.7, 0.3])
# for gradient boosted tree regression
dt_train = splits[0]
dt_test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", dt_train.count(), " Testing Rows:", dt_test.count())

dt_train.show(5)


Training Rows: 304 Testing Rows: 132
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
2020| 1| 2| 43| New York| 40.7672726|-73.97152637| New York| 0.0| 1.0| 1.0|
2020| 1| 2| 56| Spokane|47.62113146|-117.4046494|Washington| 0.0| 3.0| 3.0|
2020| 1| 2| 63|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
2020| 1| 3| 47| Spokane|47.62113146|-117.4046494|Washington| 0.0| 3.0| 3.0|
2020| 1| 3| 58|Los Angeles|34.30828379|-118.2282411|California| 0.0| 0.0| 0.0|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
only showing top 5 rows

In [0]:
from pyspark.ml.classification import RandomForestClassifier
#assembler = VectorAssembler(inputCols =["Day","Temp","Lat","Long","Admin_index","Province_index"],outputCol="normfeatures")
assembler = VectorAssembler(inputCols =["Date","Day","Year"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols =["Date","Year","Day","Temp"],outputCol="features")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
dt = RandomForestClassifier(labelCol="label", featuresCol="features",impurity="gini",featureSubsetStrategy="auto",numTrees=10,maxDepth=30,maxBins=128,seed=1234)
pipeline = Pipeline(stages=[assembler,minMax,featVect,dt])

piplineModel = pipeline.fit(dt_train)
print("Pipeline complete!") 
prediction = piplineModel.transform(dt_test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print(accuracy)

# COMMAND ----------

print("Test Error = %g" % (1.0 - accuracy))

#dt = DecisionTreeRegressor(featuresCol='features', labelCol='label', maxBins=77582)
#dt_pipeline = Pipeline(stages=[assembler, dt])
#paramGrid = ParamGridBuilder().build()
#cv = CrossValidator(estimator=dt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=5)
#dt_model = cv.fit(dt_train)
#dt_prediction = dt_model.transform(dt_test)
#dt_predicted = dt_prediction.select("features", "prediction", "trueLabel")
#dt_predicted.show(10)

#dt_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
#dt_rmse = dt_evaluator.evaluate(dt_prediction)


#print ("Root Mean Square Error (RMSE):", dt_rmse)
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#evaluation = MulticlassClassificationEvaluator(
 #   labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluation.evaluate(dt_prediction)
#print(accuracy)

Pipeline complete!
+------------------------------+----------+---------+
features |prediction|trueLabel|
+------------------------------+----------+---------+
[0.0,0.25,0.5] |0.0 |0.0 |
[0.0,0.5,0.5] |0.0 |0.0 |
[0.0,0.5,0.5] |0.0 |0.0 |
[0.0,0.75,0.5] |1.0 |1.0 |
[0.0,0.75,0.5] |1.0 |1.0 |
[0.0,1.0,0.5] |1.0 |1.0 |
[0.03333333333333333,0.25,0.5]|0.0 |0.0 |
[0.03333333333333333,0.25,0.5]|0.0 |0.0 |
[0.03333333333333333,0.75,0.5]|1.0 |1.0 |
[0.03333333333333333,1.0,0.5] |1.0 |1.0 |
[0.03333333333333333,1.0,0.5] |1.0 |1.0 |
[0.06666666666666667,0.25,0.5]|0.0 |0.0 |
[0.06666666666666667,0.25,0.5]|0.0 |0.0 |
[0.06666666666666667,0.5,0.5] |0.0 |0.0 |
[0.06666666666666667,0.75,0.5]|1.0 |1.0 |
[0.06666666666666667,0.75,0.5]|1.0 |1.0 |
[0.06666666666666667,0.75,0.5]|1.0 |1.0 |
[0.1,0.25,0.5] |0.0 |0.0 |
[0.1,0.5,0.5] |0.0 |0.0 |
[0.1,0.5,0.5] |0.0 |0.0 |
[0.1,0.5,0.5] |0.0 |0.0 |
[0.1,1.0,0.5] |1.0 |1.0 |
[0.13333333333333333,0.5,0.5] |0.0 |1.0 |
[0.13333333333333333,1.0,0.5] |1.0 |1.0 |
[0.13333333333333333,1.0,0.5] |1.0 |1.0 |
[0.16666666666666666,0.25,0.5]|0.0 |0.0 |
[0.16666666666666666,0.5,0.5] |0.0 |0.0 |
[0.16666666666666666,0.5,0.5] |0.0 |1.0 |
[0.16666666666666666,0.75,0.5]|1.0 |1.0 |
[0.16666666666666666,1.0,0.5] |1.0 |1.0 |
[0.2,0.25,0.5] |0.0 |0.0 |
[0.2,0.25,0.5] |0.0 |0.0 |
[0.2,1.0,0.5] |1.0 |1.0 |
[0.2,1.0,0.5] |1.0 |1.0 |
[0.2,1.0,0.5] |1.0 |1.0 |
[0.23333333333333334,0.25,0.5]|0.0 |0.0 |
[0.23333333333333334,0.25,0.5]|0.0 |0.0 |
[0.23333333333333334,0.25,0.5]|0.0 |0.0 |
[0.23333333333333334,0.5,0.5] |0.0 |1.0 |
[0.23333333333333334,0.75,0.5]|1.0 |1.0 |
[0.23333333333333334,0.75,0.5]|1.0 |1.0 |
[0.23333333333333334,1.0,0.5] |1.0 |1.0 |
[0.23333333333333334,1.0,0.5] |1.0 |1.0 |
[0.26666666666666666,0.75,0.5]|1.0 |1.0 |
[0.26666666666666666,1.0,0.5] |1.0 |1.0 |
[0.26666666666666666,1.0,0.5] |1.0 |1.0 |
[0.3,0.5,0.5] |0.0 |1.0 |
[0.3,0.75,0.5] |1.0 |1.0 |
[0.3333333333333333,0.25,0.5] |0.0 |0.0 |
[0.3333333333333333,0.25,0.5] |0.0 |0.0 |
[0.3333333333333333,0.25,0.5] |0.0 |0.0 |
[0.3333333333333333,0.5,0.5] |0.0 |1.0 |
[0.3333333333333333,0.5,0.5] |0.0 |1.0 |
[0.36666666666666664,0.25,0.5]|0.0 |0.0 |
[0.36666666666666664,0.25,0.5]|0.0 |0.0 |
[0.36666666666666664,0.25,0.5]|0.0 |0.0 |
[0.36666666666666664,0.5,0.5] |1.0 |0.0 |
[0.36666666666666664,0.75,0.5]|1.0 |1.0 |
[0.4,0.25,0.5] |0.0 |0.0 |
[0.4,0.5,0.5] |1.0 |1.0 |
[0.43333333333333335,0.25,0.5]|0.0 |0.0 |
[0.43333333333333335,0.25,0.5]|0.0 |0.0 |
[0.43333333333333335,0.25,0.5]|0.0 |0.0 |
[0.43333333333333335,0.25,0.5]|0.0 |0.0 |
[0.43333333333333335,0.5,0.5] |1.0 |1.0 |
[0.43333333333333335,0.75,0.5]|1.0 |1.0 |
[0.4666666666666667,0.25,0.5] |0.0 |0.0 |
[0.4666666666666667,0.5,0.5] |1.0 |1.0 |
[0.4666666666666667,0.75,0.5] |1.0 |1.0 |
[0.4666666666666667,0.75,0.5] |1.0 |1.0 |
[0.5,0.25,0.5] |0.0 |0.0 |
[0.5,0.25,0.5] |0.0 |0.0 |
[0.5,0.5,0.5] |1.0 |1.0 |
[0.5,0.5,0.5] |1.0 |1.0 |
[0.5333333333333333,0.25,0.5] |0.0 |0.0 |
[0.5333333333333333,0.25,0.5] |0.0 |0.0 |
[0.5333333333333333,0.5,0.5] |1.0 |1.0 |
[0.5333333333333333,0.75,0.5] |1.0 |1.0 |
[0.5333333333333333,0.75,0.5] |1.0 |1.0 |
[0.5333333333333333,0.75,0.5] |1.0 |1.0 |
[0.5666666666666667,0.5,0.5] |1.0 |1.0 |
[0.5666666666666667,0.75,0.5] |1.0 |1.0 |
[0.6,0.25,0.5] |0.0 |0.0 |
[0.6,0.25,0.5] |0.0 |0.0 |
[0.6,0.25,0.5] |0.0 |0.0 |
[0.6,0.5,0.5] |1.0 |1.0 |
[0.6,0.5,0.5] |1.0 |1.0 |
[0.6,0.5,0.5] |1.0 |1.0 |
[0.6,0.75,0.5] |1.0 |1.0 |
[0.6,0.75,0.5] |1.0 |1.0 |
[0.6333333333333333,0.5,0.5] |1.0 |1.0 |
[0.6666666666666666,0.5,0.5] |1.0 |1.0 |
[0.7,0.25,0.5] |0.0 |0.0 |
[0.7,0.25,0.5] |0.0 |0.0 |
[0.7,0.75,0.5] |1.0 |1.0 |
[0.7333333333333333,0.0,0.5] |0.0 |0.0 |
[0.7333333333333333,0.0,0.5] |0.0 |0.0 |
[0.7333333333333333,0.0,0.5] |0.0 |0.0 |
[0.7333333333333333,0.25,0.5] |0.0 |0.0 |
[0.7333333333333333,0.5,0.5] |1.0 |1.0 |
+------------------------------+----------+---------+
only showing top 100 rows

0.946969696969697
Test Error = 0.0530303